<a href="https://colab.research.google.com/github/cmlg96/spatial-data-management-with-GEE/blob/main/Lab04_cm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Part 1: This code filters a collection of geographic data to find records matching the name “Knox”, converts them into a DataFrame, adds a numbering column, and displays the resulting table without indexes.

In [ ]:
!pip install geemap

In [ ]:
!pip install earthengine-api

In [ ]:
import geemap
import ee

In [ ]:
ee.Authenticate()
ee.Initialize(project='my-project')

In [ ]:
# Create an interactive map
Map = geemap.Map()

# Load the counties feature collection
countries = ee.FeatureCollection("TIGER/2018/Counties")

# Add the feature collection to the map
Map.addLayer(countries, {}, 'Countries')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Define the list of names to search for
search_name = ["Knox"]

# Filter the collection by the specified names
filtered_countries = countries.filter(ee.Filter.inList("NAME", search_name))

# Convert the filtered collection to a DataFrame
df = geemap.ee_to_df(filtered_countries)

# Add a row numbering column (starting from 1)
df.insert(0, "N°", range(1, len(df) + 1))

# Print the table in a compact format
print(df.to_string(index=False))

 N°      ALAND     AWATER CBSAFP CLASSFP COUNTYFP COUNTYNS CSAFP FUNCSTAT GEOID    INTPTLAT     INTPTLON LSAD METDIVFP MTFCC NAME    NAMELSAD STATEFP
  1 1855569287    8837382  23660      H1      095 00424249              A 17095 +40.9309446 -090.2137921   06          G4020 Knox Knox County      17
  2 1336511196   20758105  47180      H1      083 00450366              A 18083 +38.6883910 -087.4203544   06          G4020 Knox Knox County      18
  3 1000523138    3743713  30940      H1      121 00516907              A 21121 +36.8874755 -083.8556297   06          G4020 Knox Knox County      21
  4  945684684 2017116243             H1      013 00581292              A 23013 +44.0420446 -069.0385145   06          G4020 Knox Knox County      23
  5 1305376626    7307094             H1      103 00758506              A 29103 +40.1368559 -092.1468345   06          G4020 Knox Knox County      29
  6 2870856403   81009708             H1      107 00835875              A 31107 +42.6344045 -097.891

#### Part 2: This code loads U.S. counties, filters Knox County in Tennessee, converts it to display, applies a red border with no fill, creates a map, centers it on the county, adds credit text, and displays it.

In [ ]:
!pip install osmnx

In [ ]:
import ee
import geemap
import osmnx as ox

In [ ]:

# Load the collection of U.S. counties
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter Knox County in Tennessee (STATEFP = 47, COUNTYFP = 093)
knox_county = counties.filter(ee.Filter.And(
    ee.Filter.eq("STATEFP", "47"),  # Filter by Tennessee
    ee.Filter.eq("COUNTYFP", "093")  # Filter by Knox County
))

# Convert the FeatureCollection to a GeoDataFrame
gdf = geemap.ee_to_gdf(knox_county)

# Convert the GeoDataFrame to an Earth Engine FeatureCollection
fc = geemap.gdf_to_ee(gdf)

# Define the style: red border with no fill
style = {
    "color": "FF0000",  # Border color (red in hexadecimal)
    "width": 2,         # Border width (2 pixels)
    "fillColor": "00000000"  # Transparent fill (no fill)
}

# Create the map
Map = geemap.Map()

# Add the layer to the map with the defined style
Map.addLayer(fc.style(**style), {}, "Knox County, Tennessee Boundary")

# Center the map on the object
Map.centerObject(fc, 10)

# Add text to the map
text = "Made by Clara MR"
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[35.99319606817822, -83.937230591891], controls=(WidgetControl(options=['position', 'transparent_bg…

#### Part 3: This code loads and filters Landsat-9 imagery for Knox County, Tennessee, applies a cloud mask, selects the least cloudy image, clips it to the county boundary, and visualizes it using the 7-6-4 band combination on an interactive map.

In [ ]:
# Load the Landsat-9 collection
landsat9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Filter the Landsat-9 collection by date and location
landsat9_filtered = landsat9.filterDate('2023-06-01', '2023-12-30') \
    .filterBounds(fc.geometry()) \
    .sort('CLOUD_COVER')  # Sort by cloud cover (ascending order)

# Function to apply cloud mask and scale the bands
def apply_cloud_mask(image):
    # Scale the optical bands
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    # Scale the thermal bands (use ST_B10)
    thermal_bands = image.select('ST_B10').multiply(0.00341802).add(149.0)
    # Apply cloud mask using the QA_PIXEL band
    qa = image.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # Cloud mask
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True).updateMask(cloud_mask)

# Apply the cloud mask to the collection
landsat9_cloud_free = landsat9_filtered.map(apply_cloud_mask)

# Select the image with the least cloud cover
best_image = landsat9_cloud_free.first()

# Clip the image using the geometry of Knox County
clipped_image = best_image.clip(fc.geometry())

# Select the band combination 7, 6, 4 (SWIR2, SWIR1, Red)
bands = ['SR_B7', 'SR_B6', 'SR_B4']
visualization = {
    'bands': bands,
    'min': 0,
    'max': 0.3,
    'gamma': 1.2
}

# Create the map
Map = geemap.Map()

# Add text to the map
text = "Made by Clara MR"
Map.add_text(text, fontsize=20, position='bottomright')

# Add the clipped image to the map
Map.addLayer(clipped_image, visualization, 'Landsat-9 Least Clouds (7-6-4)')

# Center the map on Knox County
Map.centerObject(fc, 10)

# Display the map
Map


Map(center=[35.99319606817822, -83.937230591891], controls=(WidgetControl(options=['position', 'transparent_bg…

#### Part 4: This code loads Sentinel-2 imagery, filters it by date, location, and cloud cover, creates a median mosaic, clips it to Knox County, and visualizes it using the 5-4-3 (Color Infrared) band combination on an interactive map.

In [ ]:
# Load the Sentinel-2 collection
s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2024-09-01', '2024-12-30') \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']) \
    .filterBounds(fc.geometry()) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 50) \
    .sort('CLOUDY_PIXEL_PERCENTAGE')  # Sort by cloud cover percentage (ascending order)

# Create a mosaic of all images
mosaic_s2 = s2_sr.median()  # Use the median to combine images and avoid extreme values

# Clip the mosaic using the geometry of Knox County
clipped_mosaic_s2 = mosaic_s2.clip(fc.geometry())

# Select the band combination "Color Infrared (5-4-3)"
bands = ['B5', 'B4', 'B3']

# Adjust the visualization range for the bands
visualization = {
    'bands': bands,
    'min': 0,
    'max': 3000,  # Adjust the maximum range if the value is too small
    'gamma': 1.4  # Gamma adjustment to improve contrast
}

# Create the map
Map = geemap.Map()

# Add text to the map
text = "Made by Clara MR"
Map.add_text(text, fontsize=20, position='bottomright')

# Add the clipped mosaic to the map
Map.addLayer(clipped_mosaic_s2, visualization, 'Sentinel-2 Color Infrared Mosaic (5-4-3)')

# Center the map on Knox County
Map.centerObject(fc, 10)

# Display the map
Map

Map(center=[35.99319606817822, -83.937230591891], controls=(WidgetControl(options=['position', 'transparent_bg…

#### Part 5: This code loads NAIP imagery, filters it by location and date, creates a mosaic, clips it to Knox County, and visualizes it using the N-R-G (Color Infrared) band combination on an interactive map.

In [ ]:
# Load the NAIP collection
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# Filter by location and date (Knox County, Tennessee)
naip_filtered = naip.filterBounds(fc.geometry()).filterDate('2022-01-01', '2023-12-31')

# Create a mosaic from the filtered collection
mosaic_naip = naip_filtered.mosaic()

# Clip the mosaic using the geometry of Knox County
clipped_mosaic_naip = mosaic_naip.clip(fc.geometry())

# Band combination: Color Infrared (NIR, R, G)
cir_vis = {
    'bands': ['N', 'R', 'G'],  # Bands: Near Infrared, Red, Green
    'min': 0,
    'max': 255
}

# Create the map
Map = geemap.Map()

# Add text to the map
text = "Made by Clara MR"
Map.add_text(text, fontsize=20, position='bottomright')

# Add the mosaic to the map
Map.addLayer(clipped_mosaic_naip, cir_vis, 'NAIP Color Infrared (N-R-G) - Mosaic')

# Center the map on the area of interest
Map.centerObject(fc.geometry(), 10)

# Display the map
Map

Map(center=[35.99319606817822, -83.937230591891], controls=(WidgetControl(options=['position', 'transparent_bg…